# データベースの作成

In [18]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [20]:
# データの読み込み
# data = pd.read_csv("../../gci_project_storage/mynavi_data2.csv", sep=",", encoding="UTF-8")
# 上手くいかないので、pickle
import pickle
with open('../../gci_project_storage/data.pickle', mode='rb') as f:
     data = pickle.load(f)

(46469, 224)

In [20]:
data.shape

(46469, 224)

<h3>目次：</h3><br>
・Geo Coding用のカラムの追加<br>
・データベースにぶち込む<br>

<h3>Geo Coding用のカラムの追加

In [21]:
new_colnames = ["altitude","latitude"]
df_new = pd.DataFrame(index=data.index, columns=new_colnames)
data = pd.concat([data,df_new],axis=1)
data.shape

(46469, 226)

<h3>データベースにぶち込む

In [22]:
# データベースの情報
server = 'gciteam16.database.windows.net'
database = 'mynavi-database'
username = 'gciteam16'
password = 'Password0'
port =1433
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [23]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0]) 
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Aug 29 2017 13:06:11 
	Copyright (C) 2017 Microsoft Corporation



In [24]:
# 接続エンジンの作成
engine_config = "?driver=ODBC+Driver+13+for+SQL+Server?charset=shift-jis"
db_settings = {
    "host": server,
    "database": database,
    "user": username,
    "password": password,
    "port":port,
    "config_query":engine_config
}
engine = create_engine('mssql+pyodbc://{user}:{password}@{host}:{port}/{database}{config_query}'.format(**db_settings))

In [26]:
# DBにぶち込む
table_name = "main_table"
data.to_sql(table_name, con=engine, if_exists="fail", index=False) # 8 h かかった笑

In [ ]:
# DBにぶち込む
table_name = "test_table"
data.head().to_sql(table_name, con=engine, if_exists="replace", index=False)

In [33]:
# DBの読み込み
sql_query = "SELECT TOP 10 address, traffic FROM main_table"
pd.read_sql_query(sql_query, con = engine)

,address,traffic
0,東京都文京区本郷４丁目,都営三田線春日(東京都)駅?徒歩2分丸ノ内線(池袋0荻窪)後楽園駅?徒歩4分都営大江戸線本郷...
1,東京都文京区本駒込４丁目,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分
2,東京都文京区本郷４丁目,都営三田線春日(東京都)駅?徒歩7分丸ノ内線(池袋0荻窪)本郷三丁目駅?徒歩8分
3,東京都台東区元浅草３丁目,銀座線田原町(東京都)駅?徒歩7分都営大江戸線新御徒町駅?徒歩9分
4,東京都文京区本駒込４丁目,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分千代田線 千駄木駅
5,東京都文京区本駒込４丁目,山手線田端駅?徒歩10分千代田線千駄木駅?徒歩12分南北線本駒込駅?徒歩13分
6,東京都台東区元浅草３丁目,銀座線田原町(東京都)駅?徒歩7分都営大江戸線新御徒町駅?徒歩9分
7,東京都文京区音羽１丁目,有楽町線護国寺駅?徒歩6分有楽町線江戸川橋駅?徒歩5分丸ノ内線(池袋0荻窪)茗荷谷駅?徒歩13分
8,東京都文京区音羽１丁目,有楽町線江戸川橋駅?徒歩5分有楽町線護国寺駅?徒歩6分丸ノ内線(池袋0荻窪)茗荷谷駅?徒歩13分
9,東京都台東区台東３丁目,日比谷線仲御徒町駅?徒歩2分山手線御徒町駅?徒歩5分銀座線末広町(東京都)駅?徒歩8分
